# What is ETL?



<img src="etl.png" width="800">


In [63]:
# Library to make HTTP requests (used to call the CoinGecko API)
import requests

# We'll use pandas to transform and clean the crypto data
import pandas as pd 

# SQLAlchemy provides tools to connect Python to databases
from sqlalchemy import create_engine

# We'll use datetime.now() to timestamp when we pull data
from datetime import datetime

## Extract

In [64]:
print("\n[1] Extracting data from CoinGecko API...")

url = "https://api.coingecko.com/api/v3/simple/price"
params = {
    # We want bitcoin and ethereum
    "ids": "bitcoin,ethereum",
    
    # Fiat currencies we want to compare against (USD, EUR, GBP)
    "vs_currencies": "usd,eur,gbp"
}

# Send a GET request to the CoinGecko API with our URL and parameters
response = requests.get(url, params=params)

# Convert the API response (which comes back in JSON format) into a Python dictionary
data = response.json()

# Our raw extracted data! 
print("Raw API response:")
print(data)


[1] Extracting data from CoinGecko API...
Raw API response:
{'bitcoin': {'usd': 112131, 'eur': 96159, 'gbp': 83402}, 'ethereum': {'usd': 4471.82, 'eur': 3834.88, 'gbp': 3326.12}}


## Transform

In [ ]:
print("\n[2] Transforming data...")

# Remove other fiat that we don't need
df = pd.DataFrame([{
    "timestamp": datetime.now(),
    "BTC_USD": data["bitcoin"]["usd"],
    "ETH_USD": data["ethereum"]["usd"]
}])

# 1. Add BTC-to-ETH price ratio (how many ETH one BTC is worth)
df["BTC_to_ETH_ratio"] = df["BTC_USD"] / df["ETH_USD"]

# 2. Add difference between BTC and ETH in USD
df["BTC_minus_ETH"] = df["BTC_USD"] - df["ETH_USD"]

# 3. Add average price of BTC and ETH (just for demo)
df["Average_BTC_ETH"] = (df["BTC_USD"] + df["ETH_USD"]) / 2

# 4. Add a flag column to mark if BTC is over $50,000
df["BTC_over_50k"] = df["BTC_USD"] > 50000

# -------------------------------------------------------

print("Data after transformation (USD only):")
print(df)


[2] Transforming data...
Data after transformation (USD only):
                   timestamp  BTC_USD  ETH_USD  BTC_to_ETH_ratio  \
0 2025-09-03 17:24:55.667960   112131  4471.82         25.075025   

   BTC_minus_ETH  Average_BTC_ETH  BTC_over_50k  
0      107659.18         58301.41          True  


## Load

In [ ]:
print("\n[3] Loading data into SQLite database...")

# Create a connection (engine) to a local SQLite database file.
engine = create_engine("sqlite:///crypto_prices.db")

# Save the DataFrame into a table called 'crypto_prices'
df.to_sql("crypto_prices", engine, if_exists="append", index=False)

print("Data successfully loaded into 'crypto_prices' table.")



[3] Loading data into SQLite database...
Data successfully loaded into 'crypto_prices' table.


In [ ]:
print("\n[4] Verifying database contents...")

# Take a look at contents in DB!
check = pd.read_sql(
    "SELECT * FROM crypto_prices ORDER BY timestamp DESC", 
    engine
)
print(check)

print("\nETL pipeline finished successfully!")



[4] Verifying database contents...
                    timestamp  BTC_USD  ETH_USD  BTC_to_ETH_ratio  \
0  2025-09-03 17:24:55.667960   112131  4471.82         25.075025   

   BTC_minus_ETH  Average_BTC_ETH  BTC_over_50k  
0      107659.18         58301.41             1  

ETL pipeline finished successfully!
